In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={   
    "rmse_threshold": 15
}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, NotebookSubstepRunResult, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
   
    inputs =
    [
        { STEP_NAME: "model_train", ENTITY_NAME: "bento_service" },
    ],
    outputs = 
    [
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
# run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
substep.add_metric('experiment_name', 'evaluation_metrics_example')

In [ ]:
# read inputs 
bento_step_inputs = substep.inputs(step_name="model_train")

In [ ]:
# load a bentoservice
from sinara.bentoml import load_bentoservice
from sklearn.metrics import mean_squared_error, r2_score
bento_serv = load_bentoservice(bento_step_inputs.bento_service)

In [ ]:
bento_service_version = bento_serv.service_version().split('.')[-1]
substep.add_metric("version", 
                   {'bento_service_version': bento_service_version})
print(bento_service_version)

In [ ]:
# evaluate model, check model quality on test dataset
import pandas as pd
test_data = bento_serv.test_data()
predictions = bento_serv.predict(pd.DataFrame(test_data['X']))
rmse = mean_squared_error(pd.DataFrame(test_data['Y']).values, predictions, squared=False)
r2 = r2_score(pd.DataFrame(test_data['Y']).values, predictions)
print("The root mean squared error (RMSE) on test set: {:.4f}".format(rmse))

In [ ]:
# add evaluation results to the step metrics
substep.add_metric('eval_result', {'r2': r2,
                              'rmse': rmse})

In [ ]:
# check eval result
rmse_threshold = substep_params["rmse_threshold"]
if rmse > rmse_threshold:
    raise Exception(f'RMSE is {rmse}, more than acceptable value of {rmse_threshold}')

In [ ]:
# stop spark
SinaraSpark.stop_session()